# シルバー層テーブル - ロボットイベント
![](img/silver_diagram.png)

このスクリプトは、Databricks Delta Live Tablesで**3つのシルバー層テーブル**を定義します。  
これらはブロンズ層（`bronze_robot_events`）の上に構築され、**データ品質チェック**、**クレンジング**、**スキーマのフラット化**を行い、分析に適したデータを提供します。

---

## 1. `silver_command_events`
**目的:** クリーンアップおよびフィルタリングされたロボットコマンド実行イベント。  
- **品質ルール**:
  - 無効な`duration_ms`（`> 0`）の行を削除
  - `battery_level`が`0`から`100`の範囲であることを保証
- **変換処理**:
  - `event_type = command_execution`でフィルタ
  - ネストされたフィールド（コマンド詳細、位置情報）のフラット化
  - 有効なタイムスタンプの保証
- **出力カラム**:
  - `robot_id`, `timestamp`, `x`, `y`, `command_name`, `duration_ms`, `success`, `battery_level`, `obstacle_detected`

---

## 2. `silver_sensor_data`
**目的:** クリーンアップされたロボットセンサー情報。  
- **品質ルール**:
  - `battery_level`が`0`から`100`の範囲であること
  - `battery_temp_c`が`0`から`50`の範囲であること
- **変換処理**:
  - バッテリーおよびLiDARセンサーデータの抽出
  - 有効なタイムスタンプの保証
- **出力カラム**:
  - `robot_id`, `timestamp`, `battery_level`, `battery_temp_c`, `obstacle_detected`, `scan_points`

---

## 3. `silver_combined_events`
**目的:** コマンド、センサー、位置情報を統合したテーブル。  
- **品質ルール**:
  - 無効な`duration_ms`（`> 0`）の行を削除
  - `battery_level`が`0`から`100`の範囲であること
- **変換処理**:
  - `event_type = command_execution`でフィルタ
  - コマンド＋センサー＋位置情報のフィールドをフラット化・統合
  - 有効なタイムスタンプの保証
- **出力カラム**:
  - `robot_id`, `timestamp`, `x`, `y`, `command_name`, `duration_ms`, `success`, `battery_level`, `obstacle_detected`

---

> ⚡ **ヒント:** 実行前に`CATALOG`、`BRONZE`、`SILVER`変数を環境に合わせて更新してください。

In [0]:
import dlt
from pyspark.sql.functions import col, when, to_timestamp

CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_command_events", comment="Cleaned and filtered command events")
@dlt.expect_or_drop("valid_duration", "duration_ms > 0")
@dlt.expect("battery_in_range", "battery_level BETWEEN 0 AND 100")
def silver_command_events():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    # Filter
    df = df.filter(col("event.event_type") == "command_execution")

    # Cleansing and flatten
    df = df.withColumn("duration_ms", when(col("event.command.duration_ms") > 0, col("event.command.duration_ms")).otherwise(None))
    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))

    # Timestamp filtering
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        "location.x",
        "location.y",
        col("event.command.name").alias("command_name"),
        "duration_ms",
        col("event.command.success").alias("success"),
        "battery_level",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected")
    )

In [0]:
CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_sensor_data", comment="Cleaned robot sensor data")
@dlt.expect("battery_level_in_range", "battery_level BETWEEN 0 AND 100")
def silver_sensor_data():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))
    df = df.withColumn("battery_temp_c", when((col("sensors.battery.temperature_c") >= 0) & (col("sensors.battery.temperature_c") <= 50), col("sensors.battery.temperature_c")).otherwise(None))

    df = df.filter(col("sensors.lidar.obstacle_detected").isNotNull())
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        "battery_level",
        "battery_temp_c",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected"),
        col("sensors.lidar.scan_points").alias("scan_points")
    )

In [0]:
CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_combined_events", comment="Combined robot command + sensor + location data")
@dlt.expect_or_drop("valid_duration", "duration_ms > 0")
@dlt.expect("battery_level_in_range", "battery_level BETWEEN 0 AND 100")
def silver_combined_events():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    df = df.filter(col("event.event_type") == "command_execution")

    df = df.withColumn("duration_ms", when(col("event.command.duration_ms") > 0, col("event.command.duration_ms")).otherwise(None))
    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        col("location.x").alias("x"),
        col("location.y").alias("y"),
        col("event.command.name").alias("command_name"),
        "duration_ms",
        col("event.command.success").alias("success"),
        "battery_level",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected")
    )